In [1]:
import warnings
warnings.filterwarnings('ignore')

# 1. Leer los datos de table

In [2]:
from bs4 import BeautifulSoup

with open('Table.html', 'r', encoding='utf-8') as file:
    html = file.read()

# Crear un objeto BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')

In [3]:
import re

# Lista para guardar resultados
result = []

# Variables para rastrear la jerarquía actual
current_periodo_parlamentario = None
current_periodo_anual = None
current_legislatura = None

# Función para limpiar espacios adicionales
def clean_text(text):
    # Reemplazar múltiples espacios con uno solo
    return re.sub(r'\s+', ' ', text).strip()

# Recorrer todas las filas de la tabla
for tr in soup.find_all('tr', valign='top'):
    # Verificar si contiene un Periodo Parlamentario
    font_periodo_parlamentario = tr.find('font', string=lambda text: text and 'Congreso de la República' in text)
    if font_periodo_parlamentario:
        current_periodo_parlamentario = clean_text(font_periodo_parlamentario.text)

    # Verificar si contiene un Periodo Anual de Sesiones
    font_periodo_anual = tr.find('font', string=lambda text: text and 'Período Anual de Sesiones' in text)
    if font_periodo_anual:
        current_periodo_anual = clean_text(font_periodo_anual.text)

    # Verificar si contiene una Legislatura
    font_legislatura = tr.find('font', string=lambda text: text and 'Legislatura' in text)
    if font_legislatura:
        current_legislatura = clean_text(font_legislatura.text)

    # Buscar todos los enlaces en la fila
    a_tags = tr.find_all('a', href=True)
    
    # Procesar solo si hay enlaces
    if a_tags:
        # Buscar el enlace que tenga texto (no vacío)
        link_text = None
        link = None
        
        # Si el enlace tiene texto
        for a_tag in a_tags:
            text = clean_text(a_tag.get_text())
            if text:
                link = a_tag['href']
                link_text = text
                break
        
        # Si encontramos un enlace con texto, guardarlo
        if link and link_text:
            result.append({
                'periodo_parlamentario': current_periodo_parlamentario,
                'periodo_anual': current_periodo_anual,
                'legislatura': current_legislatura,
                'descripcion': link_text,
                'link': link
            })

# Mostrar los resultados
for item in result[13:15]:
    print(f"Periodo Parlamentario: {item['periodo_parlamentario']}")
    print(f"Periodo Anual de Sesiones: {item['periodo_anual']}")
    print(f"Legislatura: {item['legislatura']}")
    print(f"Descripción: {item['descripcion']}")
    print(f"Link: {item['link']}")
    print("-" * 40)

print("Total de resultados:", len(result))

Periodo Parlamentario: Congreso de la República - Periodo Parlamentario 2021 - 2026
Periodo Anual de Sesiones: Período Anual de Sesiones 2025 - 2026
Legislatura: Primera Legislatura Ordinaria
Descripción: Asistencias y votaciones de la sesión del 10-9-2025
Link: javascript:openWindow('Apleno/864A0C3F7EE3505705258D010079B963/$FILE/Asis_y_vot_del_10-9-2025.pdf')
----------------------------------------
Periodo Parlamentario: Congreso de la República - Periodo Parlamentario 2021 - 2026
Periodo Anual de Sesiones: Período Anual de Sesiones 2025 - 2026
Legislatura: Primera Legislatura Ordinaria
Descripción: Asistencias y votaciones de la sesión del 4-9-2025
Link: javascript:openWindow('Apleno/7842EE7A61C8330405258CFB005A6739/$FILE/Asis_y_vot_del_4-9-2025.pdf')
----------------------------------------
Total de resultados: 937


In [4]:
# Debug de los datos de los links
from collections import Counter

link_types = Counter()
for item in result:
    if 'javascript:openWindow(' in item['link']:
        link_types['javascript'] += 1
    else:
        link_types['otros'] += 1

print(link_types)

Counter({'javascript': 937})


# 2. Generar los links de descarga y nombre de cada archivo

In [5]:
# --------------------------------------------
# SCRIPT PARA EXTRAER Y PROCESAR LINKS DE SESIONES DEL CONGRESO
#
# - Filtra los enlaces que usan "javascript:openWindow("
# - Extrae el link final agregando una base URL
# - Limpia y estandariza nombres de archivo
# - Genera un nombre compacto por fila con formato:
#     {índice}_pp{periodo_parlamentario}_pa{periodo_anual}_leg{número}.pdf
# - Guarda el resultado en un archivo TXT separado por ';' y codificado en utf-8
#
# Ejemplo de salida:
#     000_pp2011_2016_pa2014_2015_leg1.pdf
# --------------------------------------------

import unicodedata
import re
import pandas as pd

def clean_filename(text):
    text = unicodedata.normalize('NFKD', text)
    text = text.encode('ASCII', 'ignore').decode('utf-8')
    text = re.sub(r'[^\w\s-]', '', text)
    text = re.sub(r'\s+', '_', text)
    return text.strip().lower()

def extract_years(text):
    match = re.search(r'(\d{4})[^\d]+(\d{4})', text)
    if match:
        return f"{match.group(1)}_{match.group(2)}"
    return clean_filename(text)

def extract_periodo_anual(text):
    match = re.search(r'(\d{4})[^\d]+(\d{4})', text)
    if match:
        return f"{match.group(1)}_{match.group(2)}"
    return clean_filename(text)

# Mapear nombre ordinal a número
ORDINAL_MAP = {
    'primera': '1',
    'segunda': '2',
    'tercera': '3',
    'cuarta': '4',
    'quinta': '5',
    'sexta': '6',
    'séptima': '7',
    'septima': '7',  # sin tilde
    'octava': '8',
    'novena': '9',
    'decima': '10',
    'décima': '10'
}

def extract_legislatura_num(text):
    text_clean = clean_filename(text)
    for palabra, numero in ORDINAL_MAP.items():
        if palabra in text_clean:
            return numero
    return '0'  # valor por defecto si no se encuentra

# Filtrar los links válidos
clean_results = [item for item in result if 'javascript:openWindow(' in item['link']]

# Crear DataFrame
df_result = pd.DataFrame(clean_results)

# Extraer el link limpio
base_link = 'https://www2.congreso.gob.pe/Sicr/RelatAgenda/PlenoComiPerm20112016.nsf/'
df_result['clean_link'] = df_result['link'].str.extract(r"javascript:openWindow\('([^']+)'\)")
df_result['clean_link'] = base_link + df_result['clean_link']
df_result.drop('link', axis=1, inplace=True)

# Reset índice y generar nombre de archivo compacto
df_result.reset_index(drop=True, inplace=True)
df_result['file_name'] = df_result.apply(lambda row: 
    f"{row.name:03d}_pp{extract_years(row['periodo_parlamentario'])}_pa{extract_periodo_anual(row['periodo_anual'])}_leg{extract_legislatura_num(row['legislatura'])}.pdf", 
    axis=1)

# Seleccionar columnas finales (AGREGADA 'descripcion')
df_result = df_result[['periodo_parlamentario', 'periodo_anual', 'legislatura', 'descripcion', 'clean_link', 'file_name']]

# Guardar en archivo .txt con separador ";" y codificación utf-8
df_result.to_csv(
    'nombres_archivos.csv',
    sep=',',
    header=True,
    index=True,
    encoding='utf-8'
)
df_result

,periodo_parlamentario,periodo_anual,legislatura,descripcion,clean_link,file_name
0,Congreso de la República - Periodo Parlamentar...,Período Anual de Sesiones 2025 - 2026,Primera Legislatura Ordinaria,Asistencias y votaciones PROVISIONALES de la s...,https://www2.congreso.gob.pe/Sicr/RelatAgenda/...,000_pp2021_2026_pa2025_2026_leg1.pdf
1,Congreso de la República - Periodo Parlamentar...,Período Anual de Sesiones 2025 - 2026,Primera Legislatura Ordinaria,Asistencias y votación de la sesión del 22-10-...,https://www2.congreso.gob.pe/Sicr/RelatAgenda/...,001_pp2021_2026_pa2025_2026_leg1.pdf
2,Congreso de la República - Periodo Parlamentar...,Período Anual de Sesiones 2025 - 2026,Primera Legislatura Ordinaria,Asistencias y votación de la sesión del 16-10-...,https://www2.congreso.gob.pe/Sicr/RelatAgenda/...,002_pp2021_2026_pa2025_2026_leg1.pdf
3,Congreso de la República - Periodo Parlamentar...,Período Anual de Sesiones 2025 - 2026,Primera Legislatura Ordinaria,Asistencias y votaciones de la sesión del 10-1...,https://www2.congreso.gob.pe/Sicr/RelatAgenda/...,003_pp2021_2026_pa2025_2026_leg1.pdf
4,Congreso de la República - Periodo Parlamentar...,Período Anual de Sesiones 2025 - 2026,Primera Legislatura Ordinaria,Asistencias y votaciones de la sesión del 9-10...,https://www2.congreso.gob.pe/Sicr/RelatAgenda/...,004_pp2021_2026_pa2025_2026_leg1.pdf
...,...,...,...,...,...,...
932,Congreso de la República - Periodo Parlamentar...,Período Anual de Sesiones 2009 - 2010,Segunda Legislatura Ordinaria,Asistencia de la sesión del 15-06-2010,https://www2.congreso.gob.pe/Sicr/RelatAgenda/...,932_pp2006_2011_pa2009_2010_leg2.pdf
933,Congreso de la República - Periodo Parlamentar...,Período Anual de Sesiones 2009 - 2010,Segunda Legislatura Ordinaria,Asistencias y votaciones de la sesión del 10-0...,https://www2.congreso.gob.pe/Sicr/RelatAgenda/...,933_pp2006_2011_pa2009_2010_leg2.pdf
934,Congreso de la República - Periodo Parlamentar...,Período Anual de Sesiones 2009 - 2010,Segunda Legislatura Ordinaria,Asistencia de la sesión del 09-06-2010,https://www2.congreso.gob.pe/Sicr/RelatAgenda/...,934_pp2006_2011_pa2009_2010_leg2.pdf
935,Congreso de la República - Periodo Parlamentar...,Período Anual de Sesiones 2009 - 2010,Segunda Legislatura Ordinaria,Asistencias y votación de la sesión del 03-06-...,https://www2.congreso.gob.pe/Sicr/RelatAgenda/...,935_pp2006_2011_pa2009_2010_leg2.pdf


In [6]:
# Validar unicidad de combinación
combinaciones = df_result[['periodo_parlamentario', 'periodo_anual', 'legislatura', 'descripcion']]

if not combinaciones.duplicated().any():
    print("✅ Combinación única por fila.")
else:
    print("❌ Hay combinaciones duplicadas. Revisa los datos.")
    print("\n--- FILAS DUPLICADAS ---\n")
    
    # Mostrar todas las filas que tienen duplicados (incluyendo la original)
    duplicados = combinaciones[combinaciones.duplicated(keep=False)]
    
    # Ordenar para agrupar los duplicados juntos
    duplicados_sorted = duplicados.sort_values(by=['periodo_parlamentario', 'periodo_anual', 'legislatura', 'descripcion'])
    
    display(duplicados_sorted)
    print(f"\n📊 Total de filas con duplicados: {len(duplicados_sorted)}")
    
    # Opcional: Mostrar solo las filas que se repiten (sin la primera ocurrencia)
    print("\n--- DUPLICADOS (sin primera ocurrencia) ---\n")
    solo_duplicados = combinaciones[combinaciones.duplicated(keep='first')]
    display(solo_duplicados)
    print(f"\n📊 Total de duplicados puros: {len(solo_duplicados)}")

❌ Hay combinaciones duplicadas. Revisa los datos.

--- FILAS DUPLICADAS ---



,periodo_parlamentario,periodo_anual,legislatura,descripcion
813,Congreso de la República - Periodo Parlamentar...,Período Anual de Sesiones 2012 - 2013,Primera Legislatura Ordinaria,Asistencia y votación a la sesión del pleno de...
814,Congreso de la República - Periodo Parlamentar...,Período Anual de Sesiones 2012 - 2013,Primera Legislatura Ordinaria,Asistencia y votación a la sesión del pleno de...



📊 Total de filas con duplicados: 2

--- DUPLICADOS (sin primera ocurrencia) ---



,periodo_parlamentario,periodo_anual,legislatura,descripcion
814,Congreso de la República - Periodo Parlamentar...,Período Anual de Sesiones 2012 - 2013,Primera Legislatura Ordinaria,Asistencia y votación a la sesión del pleno de...



📊 Total de duplicados puros: 1


# 3. Descargar secuencial


In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import os
import requests

OUTPUT_PATH = '../data/scraping/a_pdfs'
os.makedirs(OUTPUT_PATH, exist_ok=True)

log_file = 'download_log.txt'

# Inicializar valores de log por defecto
last_index = 0

# Leer progreso anterior del log (si existe)
if os.path.exists(log_file):
    with open(log_file, 'r') as log:
        log_data = log.read().strip()
        if log_data.isdigit():
            last_index = int(log_data)
            print(f"Reanudando desde el índice {last_index}")
        else:
            print("Log mal formado. Iniciando desde cero.")
else:
    print("No se encontró log. Iniciando desde cero.")

# Total de archivos a descargar
total_archivos = len(df_result)

# Descargar desde el índice donde se quedó
for index, row in df_result.iloc[last_index:].iterrows():
    clean_link = row['clean_link']
    file_name = row['file_name']
    file_path = os.path.join(OUTPUT_PATH, file_name)

    print(f"[{index + 1}/{total_archivos}] Descargando {file_name} desde {clean_link}")

    try:
        response = requests.get(clean_link, verify=False)
        if response.status_code == 200:
            with open(file_path, 'wb') as f:
                f.write(response.content)
            print(f"✅ Guardado en: {file_path}")

            # Actualizar el log con el índice del próximo archivo
            with open(log_file, 'w') as log:
                log.write(str(index + 1))
        else:
            print(f"❌ Error {response.status_code} al descargar {clean_link}")
    except requests.exceptions.RequestException as e:
        print(f"⚠️ Error de red al descargar {file_name}: {e}")

    porcentaje = (index + 1) / total_archivos * 100
    print(f"📊 Progreso: {index + 1}/{total_archivos} ({porcentaje:.2f}%)")
    print('-' * 50)

# 4. Descargar paralelo

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import os
import requests
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

# 🔧 Parámetro: número de descargas simultáneas
MAX_WORKERS = 5
TIMEOUT = 200  # segundos

# 📂 Archivos de entrada
DATA_FILE = 'nombres_archivos.csv'
LOG_FILE = 'download_log.txt'
DOWNLOAD_DIR = 'downloads'

# Crear carpeta de destino si no existe
os.makedirs(DOWNLOAD_DIR, exist_ok=True)

# Leer el DataFrame desde el TXT
try:
    df_result = pd.read_csv(DATA_FILE, sep=';', encoding='utf-8')  # usa utf-8 para evitar errores de caracteres
except Exception as e:
    print(f"❌ Error al leer '{DATA_FILE}': {e}")
    exit(1)

# Verificar que existan columnas necesarias
required_cols = {'file_name', 'clean_link'}
if not required_cols.issubset(df_result.columns):
    print(f"❌ El archivo debe contener las columnas: {required_cols}")
    exit(1)

# Leer el índice de reanudación
last_index = 0
if os.path.exists(LOG_FILE):
    with open(LOG_FILE, 'r') as log:
        log_data = log.read().strip()
        if log_data.isdigit():
            last_index = int(log_data)
            print(f"🔄 Reanudando desde el índice {last_index}")
        else:
            print("⚠️ Log mal formado. Iniciando desde cero.")
else:
    print("🆕 No se encontró log. Iniciando desde cero.")

# Subconjunto pendiente por descargar
total_archivos = len(df_result)
df_to_download = df_result.iloc[last_index:]

# Función de descarga
def download_file(index, row):
    file_name = row['file_name']
    file_path = os.path.join(DOWNLOAD_DIR, file_name)
    url = row['clean_link']

    try:
        response = requests.get(url, verify=False, timeout=TIMEOUT)
        if response.status_code == 200:
            with open(file_path, 'wb') as f:
                f.write(response.content)
            return index, True, None
        else:
            return index, False, f"HTTP {response.status_code}"
    except requests.exceptions.RequestException as e:
        return index, False, str(e)

# Descarga paralela con barra de progreso
with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
    futures = {
        executor.submit(download_file, index, row): index
        for index, row in df_to_download.iterrows()
    }

    for future in tqdm(as_completed(futures), total=len(futures), desc="📥 Descargando archivos"):
        index, success, error = future.result()
        if success:
            # Guardar el índice del siguiente archivo
            with open(LOG_FILE, 'w') as log:
                log.write(str(index + 1))
        else:
            print(f"⚠️ Error en índice {index}: {error}")